In [ ]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
from functools import wraps
plt.style.use('seaborn')
from scipy.stats import norm

In [3]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

In [3]:
#Wrapper to time functions
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ti = time.time()
        result = f(*args, **kw)
        tf = time.time()
        t = tf-ti
        return result, t
    return wrap


def averageOfList(num):
    sumOfNumbers = 0
    for t in num:
        sumOfNumbers = sumOfNumbers + t

    avg = sumOfNumbers / len(num)
    return avg


In [4]:
#Lanczos algorithm, with only the ground state
@timing
def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

#brute-force full diagnolization, with all eigenvectors and eigenvalues
@timing
def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

In [8]:
lan_avg = []
full_avg = []

for i in range(11):  #here put N-1 (maximum)
    N = i+2
    alpha = 1   #density of RBM
    M = alpha*N
    # Constant A
    B = 0.95
    # Variable A
    #B=N/2
    A = N/2
    N0 = N/2
    # List of Ak
    Ak = []
    for i in range(N - 1):
        # Constant A
        #Ak_i = 1
        # Variable A
        Ak_i = A / (N0) * np.exp(-i / N0)
        Ak.append(Ak_i)
    # Define hamiltonian and hilbert space
    ha, hi = CSHam(N,B,Ak)

    #Exact Diagonalization
    #e, v = exactDiagonalization(ha)
    #Ground state energy
    #edEng = e[0]
    # Ground state
    #edState = v[0]

    #Lists for Histogram Data
    numRuns = 10
    hisIt = np.arange(numRuns)
    runTime_lan = []
    runTime_full = []

    # Cluster multiproccessing
    #ncpus = int(os.environ.get('SLURM_CPUS_PER_TASK',default=50))
    #pool = mp.Pool(processes=ncpus)
    # Run Descent
    #results_lan = [pool.apply(exactDiagonalization, args=(ha)) for x in hisIt]
    #results_full = [pool.apply(exactDiagonalization_full, args=(ha)) for x in hisIt]
    
    
    # Get errors for each run in histogram
    for i in range(len(hisIt)):
        runTime_lan_temp = exactDiagonalization(ha)[1]
        runTime_full_temp = exactDiagonalization_full(ha)[1]
        #runTime_lan_temp = results_lan[i][1]
        #runTime_full_temp = results_full[i][1]
        
        runTime_lan.append(runTime_lan_temp)
        runTime_full.append(runTime_full_temp)
        print('runTime_lan', runTime_lan_temp)
        print('runTime_full', runTime_full_temp)

        
    #average the runtime for every choice of N
    lan_avg.append(averageOfList(runTime_lan))
    full_avg.append(averageOfList(runTime_full))
    
    
#Save data to JSON file
data = [lan_avg, full_avg]
fileName = "2021_summer_data/runTime_exact_var.json"
open(fileName, "w").close()
with open(fileName, 'a') as file:
    for item in data:
        line = json.dumps(item)
        file.write(line + '\n')
print('SAVED')


runTime_lan 0.002938508987426758
runTime_full 0.0007698535919189453
runTime_lan 0.0020797252655029297
runTime_full 0.0005509853363037109
runTime_lan 0.0019412040710449219
runTime_full 0.0005533695220947266
runTime_lan 0.0018544197082519531
runTime_full 0.0005373954772949219
runTime_lan 0.0018277168273925781
runTime_full 0.0005278587341308594
runTime_lan 0.0018041133880615234
runTime_full 0.0005109310150146484
runTime_lan 0.0017380714416503906
runTime_full 0.0005218982696533203
runTime_lan 0.0020780563354492188
runTime_full 0.0005338191986083984
runTime_lan 0.0017504692077636719
runTime_full 0.0004899501800537109
runTime_lan 0.0016961097717285156
runTime_full 0.0004878044128417969
runTime_lan 0.0015664100646972656
runTime_full 0.0005395412445068359
runTime_lan 0.0019888877868652344
runTime_full 0.0005097389221191406
runTime_lan 0.0019445419311523438
runTime_full 0.0005233287811279297
runTime_lan 0.0019373893737792969
runTime_full 0.0005095005035400391
runTime_lan 0.0018994808197021484
r

In [5]:
N = 10
Ak = []

alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.875
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
e, v = exactDiagonalization_full(ha)


In [6]:
print(e)

(array([-1.41524943, -1.32771232, -1.27145158, -1.21988005, -1.19876478,
       -1.11306153, -1.11089347, -1.0858455 , -1.07493451, -0.98756256,
       -0.93938085, -0.9338297 , -0.91135368, -0.90173021, -0.8935346 ,
       -0.8730331 , -0.80315108, -0.69343838, -0.62406881, -0.53438139,
       -0.50864167, -0.38721415, -0.36979703, -0.31053015, -0.25413804,
       -0.18236887, -0.17478303, -0.09929506, -0.04515056,  0.04131016,
        0.10087815,  0.24035625,  0.35185493,  0.41431648,  0.41522106,
        0.43187103,  0.44636995,  0.45165595,  0.4549342 ,  0.46446678,
        0.47988562,  0.51981148,  0.5259888 ,  0.53022274,  0.53638579,
        0.59887303,  0.65252521,  0.6527589 ,  0.69325189,  0.71139475,
        0.71240188,  0.73158247,  0.74761339,  0.75153445,  0.78431651,
        0.86470533,  0.8726142 ,  0.88383632,  0.88996922,  0.97406376,
        0.99132177,  1.03767668,  1.09885071,  1.19035625]), [array([ 0.00000000e+00+0.j,  5.07439612e-02+0.j,  7.23437122e-02+0.j,
   